In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import pandas
import utils
from data_processing_bureaucrat.Bureaucrat import Bureaucrat
from pathlib import Path
import plotly.express as px
import measurements_table
from calculate_interpixel_distance import script_core as calculate_interpixel_distance

Before starting, we build a table relating each scan name with each "voltage sweep measurement"

In [ ]:
measurements_table_df = measurements_table.create_measurements_table()

# For each of the "scan 1D seeping bias voltage" I collect all the single voltage 1D scans that were created in such "voltage sweep scan".
scans_and_sub_measurements_df = pandas.DataFrame()
for root_measurement_name in measurements_table_df.query('Type=="scan 1D sweeping bias voltage"').index:
    with open(utils.path_to_measurements_directory/Path(root_measurement_name)/Path('scan_1D_sweeping_bias_voltage/README.txt'), 'r') as ifile:
        for idx, line in enumerate(ifile):
            if idx == 0:
                continue
            scans_and_sub_measurements_df = scans_and_sub_measurements_df.append({'Scan name': line.replace('\n',''), 'Voltage scan measurement name': root_measurement_name}, ignore_index=True)
scans_and_sub_measurements_df.set_index('Scan name', inplace=True)
display(scans_and_sub_measurements_df)

Read interpixel distance and calibration factor (previously calculated by the respective scripts) for each measurement:

In [ ]:
interpixel_distances_df = pandas.DataFrame()
for measurement_name in measurements_table_df.query('Type=="scan 1D"').index:
    try:
        this_measurement_IPD = utils.read_previously_calculated_inter_pixel_distance(measurement_name)
    except FileNotFoundError:
        this_measurement_IPD = float('NaN')
        print(f'Cannot find IPD for measurement {measurement_name}...')
    try:
        this_measurement_calibration_factor = utils.read_previously_calculated_distance_calibration_factor(measurement_name)
    except FileNotFoundError:
        this_measurement_calibration_factor = float('NaN')
        print(f'Cannot find distance calibration factor for measurement {measurement_name}...')
    try:
        this_measurement_belongs_to_the_voltage_scan = scans_and_sub_measurements_df.loc[measurement_name,'Voltage scan measurement name']
    except KeyError:
        this_measurement_belongs_to_the_voltage_scan = '?'
    interpixel_distances_df = interpixel_distances_df.append(
        {
            'Measurement name': measurement_name,
            'IPD (m)': this_measurement_IPD,
            'Distance calibration factor':this_measurement_calibration_factor,
            'Voltage scan measurement name': this_measurement_belongs_to_the_voltage_scan,
        },
        ignore_index = True,
    )
interpixel_distances_df.set_index('Measurement name', inplace=True)
display(interpixel_distances_df)

Now build a full dataframe with this information and the information of each detector:

In [ ]:
interpixel_distances_df['Device'] = measurements_table_df['Measured device']
interpixel_distances_df['Bias voltage (V)'] = measurements_table_df['Bias voltage (V)']
interpixel_distances_df.loc[interpixel_distances_df['Bias voltage (V)']=='?','Bias voltage (V)'] = 'NaN'
interpixel_distances_df['Bias voltage (V)'] = interpixel_distances_df['Bias voltage (V)'].astype(float)
interpixel_distances_df.reset_index(inplace=True)
interpixel_distances_df.set_index('Device', inplace=True)
interpixel_distances_df = interpixel_distances_df.join(utils.bureaucrat.devices_sheet_df)

display(interpixel_distances_df)
print(sorted(interpixel_distances_df.columns))

In [ ]:
df = interpixel_distances_df.copy().reset_index()
df = df.query('`Bias voltage (V)`<230') # Higher values produce amplitudes > 1 V and enter into the nonlinear regime of the Cividec amplifiers.
df = df[~df['Device'].isin({'1','2'})]
df = df[~df['Voltage scan measurement name'].isin(
    {
        '20211024163128_#65_sweeping_bias_voltage', # This measurement did not have all the pads grounded.
    }
)]
df['IPD with calibration (m)'] = df['IPD (m)']*df['Distance calibration factor']
df = df.sort_values(by=['Bias voltage (V)','trenches','trench depth'])
fig = utils.line(
    data_frame = df,
    line_group = 'Voltage scan measurement name',
    x = 'Bias voltage (V)',
    y = 'IPD with calibration (m)',
    facet_col = 'wafer',
    facet_row = 'trenches',
    color = 'trench depth',
    symbol = 'pixel border',
    line_dash = 'contact type',
    grouped_legend = True,
    hover_name = 'Measurement name',
    text = 'Device',
)
fig.show()

In [ ]:
fig.write_html('ipd_vs_bias_voltage.html', include_plotlyjs = 'cdn')